In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [2]:
# Create and register a new `tqdm` instance with `pandas`
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

In [3]:
interaction_action_types = {
    'clickout item',
    'interaction item rating',
    'interaction item info',
    'interaction item image',
    'interaction item deals',
    'search for item'
}

In [4]:
df = pd.read_csv('../data/validation.csv').set_index(keys=['session_id', 'step'])

In [11]:
# df[df['session_id'] == '0570c0ad1a064']
df.head(100)

user_id   timestamp             action_type  \
session_id    step                                                     
ad89aba9bb002 1     0BO8K084XKYE  1541457675         search for item   
              2     0BO8K084XKYE  1541458697  interaction item image   
              3     0BO8K084XKYE  1541458697  interaction item image   
              4     0BO8K084XKYE  1541458705           clickout item   
              5     0BO8K084XKYE  1541458805           clickout item   
74986cb514e04 1     0CI40GAHP96Q  1541456132  search for destination   
              2     0CI40GAHP96Q  1541456168           clickout item   
              3     0CI40GAHP96Q  1541456747  search for destination   
              4     0CI40GAHP96Q  1541456767  search for destination   
              5     0CI40GAHP96Q  1541456787    change of sort order   
              6     0CI40GAHP96Q  1541456826  search for destination   
              7     0CI40GAHP96Q  1541456884           clickout item   
0570c0ad1a064 1     0OQU0SD8LPT3  1541455534  interaction item image   
              2     0OQU0SD8LPT3  1541455534  interaction item image   
              3     0OQU0SD8LPT3  1541455534  interaction item image   
              4     0OQU0SD8LPT3  1541455534  interaction item image   
              5     0OQU0SD8LPT3  1541455534  interaction item image   
              6     0OQU0SD8LPT3  1541455534  interaction item image   
              7     0OQU0SD8LPT3  1541455534  interaction item image   
              8     0OQU0SD8LPT3  1541455544  interaction item image   
              9     0OQU0SD8LPT3  1541455544  interaction item image   
              10    0OQU0SD8LPT3  1541455544  interaction item image   
              11    0OQU0SD8LPT3  1541455544  interaction item image   
              12    0OQU0SD8LPT3  1541455544  interaction item image   
              13    0OQU0SD8LPT3  1541455544  interaction item image   
              14    0OQU0SD8LPT3  1541455544  interaction item image   
              15    0OQU0SD8LPT3  1541455544  interaction item image   
              16    0OQU0SD8LPT3  1541455544  interaction item image   
              17    0OQU0SD8LPT3  1541455544  interaction item image   
              18    0OQU0SD8LPT3  1541455544  interaction item image   
...                          ...         ...                     ...   
f700004a9608f 18    2I3VJP0O4QN9  1541456320  interaction item image   
              19    2I3VJP0O4QN9  1541456320  interaction item image   
              20    2I3VJP0O4QN9  1541456320  interaction item image   
              21    2I3VJP0O4QN9  1541456320  interaction item image   
              22    2I3VJP0O4QN9  1541456320  interaction item image   
              23    2I3VJP0O4QN9  1541456320  interaction item image   
              24    2I3VJP0O4QN9  1541456330  interaction item image   
              25    2I3VJP0O4QN9  1541456334           clickout item   
              26    2I3VJP0O4QN9  1541456853           clickout item   
b46b69b053ea4 1     2I49QL4EG4ZC  1541456105         search for item   
              2     2I49QL4EG4ZC  1541456108         search for item   
              3     2I49QL4EG4ZC  1541456169         search for item   
              4     2I49QL4EG4ZC  1541456313  search for destination   
              5     2I49QL4EG4ZC  1541456543         search for item   
              6     2I49QL4EG4ZC  1541456591        filter selection   
              7     2I49QL4EG4ZC  1541456591  search for destination   
              8     2I49QL4EG4ZC  1541456601        filter selection   
              9     2I49QL4EG4ZC  1541456621         search for item   
              10    2I49QL4EG4ZC  1541456671  search for destination   
              11    2I49QL4EG4ZC  1541456878         search for item   
              12    2I49QL4EG4ZC  1541456935           clickout item   
9b028a29d99b1 1     2N2RU0NDDZ88  1541456259  search for destination   
              2     2N2RU0NDDZ88  1541456349  interaction item im

In [6]:
def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out


def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

In [7]:
df_target = get_submission_target(df)
df_target

,,user_id,timestamp,action_type,reference,platform,city,device,current_filters,impressions,prices
session_id,step,,,,,,,,,,
ad89aba9bb002,5,0BO8K084XKYE,1541458805,clickout item,NaN,US,"Ledyard, USA",tablet,NaN,76028|70388|1336053|317126|79506|9979396|71272...,81|70|104|153|100|86|121|113|113|94|39|94|75|4...
74986cb514e04,7,0CI40GAHP96Q,1541456884,clickout item,NaN,BR,"Rio Quente, Brazil",mobile,NaN,1668387|2260908|3926372|2343438|2571941|257499...,49|30|31|60|354|280|33|50|307|56|64|273|52|27|40
0570c0ad1a064,27,0OQU0SD8LPT3,1541455563,clickout item,NaN,IT,"Leukerbad, Switzerland",mobile,NaN,28940|351361|28939|28930|28947|28926|28927|289...,225|177|424|161|201|194|153|175|157|204|159|33...
86052262c2b01,31,101QC20YANDM,1541456080,clickout item,NaN,UK,"Bristol, United Kingdom",tablet,NaN,12558|12563|12564|12568|12565|46606|47804|1257...,191|189|170|136|160|131|91|168|128|221|170|113...
957660f1348c1,5,1YXNJH0CVWD1,1541459188,clickout item,NaN,BR,"São Paulo, Brazil",mobile,Sort by Price,6703430|8136104|2707078|6935282|3145604|450574...,11|12|14|16|17|17|17|17|17|17|18|18|18|18|18|1...
e7ae0d9b9548b,4,2GUABJ97DL44,1541457420,clickout item,NaN,RU,"Rybinsk, Russia",mobile,NaN,2852220|1710581|2526176|4969070|2037901|321307...,76|19|29|28|37|56|36|31|20|23|32|34|19|40|133|...
10f6a151699ca,1,2HMIJDJ4O1SM,1541457984,clickout item,NaN,IE,"Dublin, Ireland",tablet,NaN,12509|12510|46119|16674|46149|640521|895195|12...,479|429|243|265|269|187|224|464|210|224|228|38...
f700004a9608f,26,2I3VJP0O4QN9,1541456853,clickout item,NaN,EC,"Fort Myers, USA",desktop,Sort by Price,63381|1416578|6174504|2518114|6297618|6946984|...,36|36|42|43|44|44|44|44|45|45|46|51|51|52|52|5...
b46b69b053ea4,12,2I49QL4EG4ZC,1541456935,clickout item,NaN,US,"Nashville, USA",mobile,NaN,75291|9293590|59561|78737|72439|9112834|956532...,155|259|117|57|220|175|350|210|249|119|284|219...


In [8]:
# index test dataset by (session_id, step)
# 
# for each action which has to be predicted: apply
#     find all interaction actions before the step of the action to predict
#     compute frequencies of items based on how many interaction actions reference them
#     sort the frequencies in descending order
#     explode the target action's impressions list
#     sort the impressions list by decending frequency order, where the default frequency is zero (the sort is stable)
#     combine the sorted impressions list into a string again
# pick only submission columns
# save csv

def sort_array_by_frequencies(array, frequencies):
    return sorted(array, key=lambda x: frequencies[x], reverse=True)

def sort_impressions_by_most_interactions(target_action):
    df_session = df.loc[target_action['session_id'], :].reset_index()
    df_before_target = df_session[(df_session['step'] < target_action['step']) & (df_session['reference'].notnull())]
    df_interactions = df_before_target[df_before_target['action_type'].isin(interaction_action_types)]

    sorted_impressions = sort_array_by_frequencies(
        array=string_to_array(target_action['impressions']),
        frequencies=Counter(df_interactions['reference'])
    )

    return ' '.join(sorted_impressions)

In [9]:
df_submission = df_target.reset_index()[["user_id", "session_id", "timestamp", "step", "impressions"]]

In [11]:
df_submission['new_impressions'] = df_submission.progress_apply(sort_impressions_by_most_interactions, axis=1)
df_submission

,user_id,session_id,timestamp,step,impressions,new_impressions
0,004A07DM0IDW,1d688ec168932,1541555799,7,2059240|2033381|1724779|127131|399441|103357|1...,1050068|2059240|3241426|2033381|1724779|127131...
1,009RGHI3G9A3,f05ab0de907e2,1541570940,2,10884872|7065316,7065316|10884872
2,00Y1Z24X8084,26b6d294d66e7,1541651823,2,2714480|4476010|3843244|3833012|9017890|198100...,3843244|2714480|4476010|3833012|9017890|198100...
3,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,3565720|2947584|4115018|2039671|3836538|801409...,2817590|3565720|2947584|4115018|2039671|383653...
4,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,1451247|559056|1045096|1963879|693596|1967173|...,7304020|1177554|1451247|559056|1045096|1963879...
5,0339C84S24ET,89171d441a304,1541615683,36,13361|5647680|116764|898719|8276346|9168|19325...,37717|4957186|13361|5647680|116764|898719|8276...
6,0386OH8JDE1Q,e09591d07cdef,1541620536,2,1193320|5488246|3858774|4552034|10620372|22696...,5488246|1193320|3858774|4552034|10620372|22696...
7,03LTH89QY623,7663406cf586c,1541554183,4,241961|906477|991561|353701|1149665|77258|4943...,59566|241961|906477|991561|353701|1149665|7725...
8,03VT0ODUTZB0,725e8adf70e86,1541632490,23,109938|164193|632366|1362450|1070666|164220|11...,1070666|322491|109938|164193|632366|1362450|16...
9,03XH0JWCWHAM,73f4c417ff730,1541566143,176,42692|5116230|42876|4342578|42864|3148690|2123...,42692|6298428|42858|3148690|42618|6452022|5762...


In [12]:
df_submission.drop(columns=['impressions']).rename(columns={'new_impressions': 'item_recommendations'}).to_csv('../data/submission_most_interactions_2.csv', index=False)

# Hypothesis: Users interact most with the items which they afterwards click out

Thus, **interaction** with items correlates with them being clicked out. We can measure **interaction** in two way:
1. The number of actions which reference an item.
2. The time duration spent interacting with an item.

## Action types with item interaction - the reference value is the item ID

- `clickout item`: user makes a click-out on the item and gets forwarded to a partner website. Other items that were displayed to the user and their associated prices are listed under the ‘impressions’ and ‘prices’ column for this action.
- `interaction item rating`: user interacts with a rating or review of an item.
- `interaction item info`: user interacts with item information.
- `interaction item image`: user interacts with an image of an item.
- `interaction item deals`: user clicks on the view more deals button.
- `search for item`: user searches for an accommodation.